In [1]:
import os
import pandas as pd
import numpy as np
from convert_eprime import convert as ep

In [2]:
source_dir = os.path.join('..','sourcedata')
derivs_dir = os.path.join('..','derivatives')

#### Read all the subject data

Reading only data for the full sample (400-series). Executing  EPrime text-to-csv conversion. Setting up for subject-level analysis.

In [3]:
for s in os.listdir(source_dir):
    if s.startswith('sub-4'):
        sub_dir = os.path.join(source_dir,s)
        for f in os.listdir(sub_dir):
            if f.endswith('.txt') and f.split('_')[-3] == 'task-procspeed':
                #print(f)
                o = f.split('.')[0]+'.csv'
                fpath = os.path.join(sub_dir,f)
                opath = os.path.join(sub_dir,o)
                if os.path.isfile(opath):
                    print(opath,'exists')
                else:
                    o = os.path.basename(fpath).split('.')[0]+'.csv' 
                    opath = os.path.join(sub_dir,o)
                    print(fpath,'=>',opath)
                ep.text_to_csv(fpath,opath)

..\sourcedata\sub-401\sub-401_task-procspeed_run-1_beh.csv exists
Output file successfully created- ..\sourcedata\sub-401\sub-401_task-procspeed_run-1_beh.csv
..\sourcedata\sub-402\sub-402_task-procspeed_run-1_beh.csv exists
Output file successfully created- ..\sourcedata\sub-402\sub-402_task-procspeed_run-1_beh.csv
..\sourcedata\sub-403\sub-403_task-procspeed_run-1_beh.csv exists
Output file successfully created- ..\sourcedata\sub-403\sub-403_task-procspeed_run-1_beh.csv
..\sourcedata\sub-404\sub-404_task-procspeed_run-1_beh.csv exists
Output file successfully created- ..\sourcedata\sub-404\sub-404_task-procspeed_run-1_beh.csv
..\sourcedata\sub-405\sub-405_task-procspeed_run-1_beh.csv exists
Output file successfully created- ..\sourcedata\sub-405\sub-405_task-procspeed_run-1_beh.csv
..\sourcedata\sub-406\sub-406_task-procspeed_run-1_beh.csv exists
Output file successfully created- ..\sourcedata\sub-406\sub-406_task-procspeed_run-1_beh.csv
..\sourcedata\sub-407\sub-407_task-procspeed_r

# Combine subjects and output trial-level

## Read in and concatenate subject datasheets

In [4]:
cleaned_dataframes_list = []

for s in os.listdir(source_dir):
    if s.startswith('sub-4'):
        sub_dir = os.path.join(source_dir,s)
        for f in os.listdir(sub_dir):
            if f.endswith('.csv') and f.split('_')[-3] == 'task-procspeed':
                fpath = os.path.join(sub_dir,f)
                df = pd.read_csv(fpath)
                cleaned_dataframes_list.append(df)

### ProcSpd data
Cleanup: Rehomogenize subject column, concatenate all frames, drop unneeded columns and rows

In [6]:
procspd_trials = pd.concat(cleaned_dataframes_list,sort=False)

In [7]:
procspd_trials = procspd_trials[procspd_trials['Procedure']=='TrialProc']
procspd_trials = procspd_trials[[
    'Subject','TargetStimulus.RT','TargetStimulus.OnsetToOnsetTime','Buffer.RT'
]]

###### Combine response windows for final RT

In [10]:
def calculate_rt(row):
    initial = row['TargetStimulus.RT']
    buffert = row['Buffer.RT']
    initial_duration = row['TargetStimulus.OnsetToOnsetTime']
    if initial == 0 and buffert > 0:
        rt = buffert + initial_duration
    elif initial > 0: rt = initial
    else: rt = np.nan
    return(rt)

procspd_trials['RT'] = procspd_trials.apply(calculate_rt,axis=1)

#### *Output ProcSpd trial-level data*

In [12]:
fpath = os.path.join(derivs_dir,'procspd_trial_level.csv')
procspd_trials.to_csv(fpath,index=False)

#### *Group and output ProcSpd subject-level data*

In [15]:
grouped = procspd_trials.groupby('Subject')
procspd_subs = grouped.mean()[['RT']]
procspd_subs.head()

,RT
Subject,
401,344.400000
402,319.000000
403,345.450000
404,511.450000
405,288.833333


In [16]:
fpath = os.path.join(derivs_dir,'procspd_subject_level.csv')
procspd_subs.to_csv(fpath)